In [2]:
import pandas as pd
import matplotlib.pyplot as plt


from collections import Counter
import re
import string


#url_testDATA='https://drive.google.com/file/d/1q8_G19zYuD4C2l8u4CrGe_yiitjI1h2I/view?usp=sharing'
url_trainDATA='https://drive.google.com/file/d/1HqD1LueAC1We0Bgum4qNxxCU6VJZW3yW/view?usp=sharing'

#https://www.kaggle.com/datatattle/email-classification-nlp?select=SMS_train.csv
url=url_trainDATA
file_id=url.split('/')[-2]
print('LOG file id = ',file_id)
dwn_url='https://drive.google.com/uc?id=' + file_id
print('LOG download url = ', dwn_url)

df = pd.read_csv(dwn_url, encoding = "ISO-8859-1")

LOG file id =  1HqD1LueAC1We0Bgum4qNxxCU6VJZW3yW
LOG download url =  https://drive.google.com/uc?id=1HqD1LueAC1We0Bgum4qNxxCU6VJZW3yW


In [3]:
print(df.head())

   S. No.                                       Message_body     Label
0       1                         Rofl. Its true to its name  Non-Spam
1       2  The guy did some bitching but I acted like i'd...  Non-Spam
2       3  Pity, * was in mood for that. So...any other s...  Non-Spam
3       4               Will ü b going to esplanade fr home?  Non-Spam
4       5  This is the 2nd time we have tried 2 contact u...      Spam


In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

In [14]:
print('\ntext\n')
data = df['Message_body']
print(data)
print('\nlabel\n')
label = df['Label']
print(label)


text

0                             Rofl. Its true to its name
1      The guy did some bitching but I acted like i'd...
2      Pity, * was in mood for that. So...any other s...
3                   Will ü b going to esplanade fr home?
4      This is the 2nd time we have tried 2 contact u...
                             ...                        
952    hows my favourite person today? r u workin har...
953                        How much you got for cleaning
954    Sorry da. I gone mad so many pending works wha...
955                                   Wat time ü finish?
956                      Just glad to be talking to you.
Name: Message_body, Length: 957, dtype: object

label

0      Non-Spam
1      Non-Spam
2      Non-Spam
3      Non-Spam
4          Spam
         ...   
952    Non-Spam
953    Non-Spam
954    Non-Spam
955    Non-Spam
956    Non-Spam
Name: Label, Length: 957, dtype: object


In [13]:
def tokenize(sequence):
    tokens = tokenizer.encode_plus(sequence, max_length=512,
                                   truncation=True, padding='max_length',
                                   add_special_tokens=True, return_token_type_ids=False,
                                   return_tensors='tf')
    return tokens['input_ids'], tokens['attention_mask']

In [31]:
import numpy as np
num_of_elements = 5000

Xids = np.zeros((num_of_elements, 512))
Xmask = np.zeros((num_of_elements, 512))

idx = np.random.randint(0, 250, num_of_elements)  
small_dataset = np.array(data)[idx.astype(int)]
small_dataset_labels = np.array(label)[idx.astype(int)]

labels = np.array(small_dataset_labels)

In [32]:
for i, sequence in enumerate(small_dataset):
    tokens = tokenize(sequence)
    Xids[i, :], Xmask[i, :] = tokens[0], tokens[1]

In [33]:
Xids[0]

array([  101.,   146.,  1354.,   178.,   112.,   173.,  1243.,  1140.,
         170.,  2824.,   117.,  1198.,  1884.,  1116.,  1115.,  1116.,
        1103.,  1912.,  1104.,  1645.,   190.,  1243.,  1527.,  1389.,
       15292.,  1582.,   119.,  1262.,  1119.,  7871.,  1177.,  1277.,
         106.,   102.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
      

In [34]:
from transformers import TFAutoModel

In [35]:
bert = TFAutoModel.from_pretrained('bert-base-cased')

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [22]:
bert.summary()

Model: "tf_bert_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  108310272 
                                                                 
Total params: 108,310,272
Trainable params: 108,310,272
Non-trainable params: 0
_________________________________________________________________


In [23]:
import tensorflow as tf

input_ids = tf.keras.layers.Input(shape=(512,), name='Message_body', dtype='int32')
mask = tf.keras.layers.Input(shape=(512,), name='Label', dtype='int32')

embeddings = bert.bert(input_ids, attention_mask=mask)[1]  # we access the transformer model within our bert object using the bert attribute (eg bert.bert instead of bert)

# Classifier head
x = tf.keras.layers.Dense(1024, activation ='relu')(embeddings)
y = tf.keras.layers.Dense(1, activation ='sigmoid', name='outputs')(x)

model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)

model.layers[2].trainable = False

In [24]:
bert.bert(input_ids, attention_mask=mask) # outputs of bert

TFBaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=<KerasTensor: shape=(None, 512, 768) dtype=float32 (created by layer 'bert')>, pooler_output=<KerasTensor: shape=(None, 768) dtype=float32 (created by layer 'bert')>, past_key_values=None, hidden_states=None, attentions=None, cross_attentions=None)

In [25]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Message_body (InputLayer)      [(None, 512)]        0           []                               
                                                                                                  
 Label (InputLayer)             [(None, 512)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  108310272   ['Message_body[0][0]',           
                                thPoolingAndCrossAt               'Label[0][0]']                  
                                tentions(last_hidde                                               
                                n_state=(None, 512,                                           

In [26]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=["accuracy"])

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

history = model.fit(
    [Xids, Xmask], labels,
    validation_split=0.8,
    batch_size = 16,
    verbose = 1,
    epochs=1)

63/63 [==============================] - 2584s 48s/step - loss: 0.9187 - accuracy: 0.5435 - val_loss: 0.6262 - val_accuracy: 0.7832